In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import LabelEncoder

In [2]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
pizza_df = pd.read_excel('Pizza_Sale.xlsx', engine='openpyxl')
ingred_df = pd.read_excel('Pizza_ingredients.xlsx', engine='openpyxl')

In [4]:
pizza_df.dropna(inplace=True)

In [9]:
def parse_dates(date):
  for fmt in ('%d-%m-%Y', '%d/%m/%Y'):
    try:
      return pd.to_datetime(date, format=fmt)
    except ValueError:
      pass
  raise ValueError(f'no valid date format found for {date}')

In [11]:
pizza_df['order_date'] = pizza_df['order_date'].apply(parse_dates)

In [12]:
pizza_df = pizza_df[['order_date', 'pizza_name', 'quantity']]

In [13]:
# pizza_df['pizza_name'].unique()

In [14]:
sales_summary = pizza_df.groupby(['order_date', 'pizza_name']).sum().reset_index()

In [15]:
sales_summary.head()

,order_date,pizza_name,quantity
0,2015-01-01,The Barbecue Chicken Pizza,11
1,2015-01-01,The Big Meat Pizza,5
2,2015-01-01,The Calabrese Pizza,1
3,2015-01-01,The California Chicken Pizza,4
4,2015-01-01,The Chicken Alfredo Pizza,2


In [21]:
# reshape data for time series modeling
sales_pivot = sales_summary.pivot(index='order_date', columns='pizza_name', values='quantity').fillna(0)

In [23]:
arima_models = {}

for pizza_name in sales_pivot.columns:
  try:
    model = ARIMA(sales_pivot[pizza_name], order=(1, 1, 0))
    model_fit = model.fit()
    arima_models[pizza_name] = model_fit
  except:
    print(f'ARIMA model for {pizza_name} failed to fit')

In [24]:
# Generate predictions for one week
prediction_days = 7
predictions_arima = {}

for pizza_name, model in arima_models.items():
  predictions_arima[pizza_name] = model.predict(start=len(sales_pivot), end=len(sales_pivot) + prediction_days - 1)

In [27]:
predictions_df = pd.DataFrame(predictions_arima)

In [29]:
predictions_df.index = pd.date_range(start=sales_pivot.index[-1], periods=prediction_days, freq='D')

In [31]:
ingredients_df = ingred_df[['pizza_name', 'pizza_ingredients', 'Items_Qty_In_Grams']]

In [33]:
# change column name to items_qty
ingredients_df.rename(columns={'Items_Qty_In_Grams': 'items_qty'}, inplace=True)

In [35]:
predictions_df

,The Barbecue Chicken Pizza,The Big Meat Pizza,The Brie Carre Pizza,The Calabrese Pizza,The California Chicken Pizza,The Chicken Alfredo Pizza,The Chicken Pesto Pizza,The Classic Deluxe Pizza,The Five Cheese Pizza,The Four Cheese Pizza,...,The Prosciutto and Arugula Pizza,The Sicilian Pizza,The Soppressata Pizza,The Southwest Chicken Pizza,The Spicy Italian Pizza,The Spinach Pesto Pizza,The Spinach Supreme Pizza,The Spinach and Feta Pizza,The Thai Chicken Pizza,The Vegetables + Vegetables Pizza
2015-12-31,8.177936,4.359358,0.888944,1.432509,7.0,2.416310,2.102540,7.119636,2.904758,7.036754,...,3.440253,2.287110,2.033516,5.917653,6.042423,3.472261,2.0,3.171528,3.0,7.835581
2016-01-01,9.007915,5.256593,1.506167,2.251518,7.0,3.252335,3.002629,8.502385,4.002268,8.000338,...,4.251190,3.759064,2.500562,5.496609,7.292032,4.250256,2.0,4.007356,3.0,8.772523
2016-01-02,8.629846,4.765913,1.163282,1.823589,7.0,2.811000,2.575658,7.838582,3.427381,7.527399,...,3.829571,2.960415,2.274866,5.689795,6.764059,3.854065,2.0,3.625284,3.0,8.366936
2016-01-03,8.802063,5.034256,1.353764,2.047181,7.0,3.043979,2.778198,8.157248,3.728513,7.759523,...,4.048777,3.393744,2.383932,5.601157,6.987133,4.055824,2.0,3.799936,3.0,8.542508
2016-01-04,8.723615,4.887505,1.247946,1.930355,7.0,2.920990,2.682120,8.004269,3.570777,7.645594,...,3.934809,3.158629,2.331226,5.641826,6.892882,3.953079,2.0,3.720099,3.0,8.466505
2016-01-05,8.759349,4.967760,1.306731,1.991396,7.0,2.985916,2.727696,8.077708,3.653400,7.701512,...,3.994063,3.286197,2.356696,5.623166,6.932704,4.005401,2.0,3.756594,3.0,8.499406
2016-01-06,8.743072,4.923870,1.274074,1.959502,7.0,2.951642,2.706076,8.042453,3.610121,7.674067,...,3.963256,3.216982,2.344388,5.631728,6.915879,3.978756,2.0,3.739912,3.0,8.485164


In [37]:
ingredients_df.head()

,pizza_name,pizza_ingredients,items_qty
0,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,The Barbecue Chicken Pizza,Red Peppers,15.0
2,The Barbecue Chicken Pizza,Green Peppers,20.0
3,The Barbecue Chicken Pizza,Tomatoes,30.0
4,The Barbecue Chicken Pizza,Red Onions,60.0


In [39]:

# Create a dictionary to store the ingredient quantities
ingredient_quantities = {}

# Iterate through each pizza in the predictions
for pizza_name in predictions_df.columns:
  # Get the predicted quantity for the pizza
  predicted_quantity = predictions_df[pizza_name].sum()

  # Get the ingredients for the pizza
  pizza_ingredients = ingredients_df[ingredients_df['pizza_name'] == pizza_name]

  # Iterate through each ingredient for the pizza
  for index, row in pizza_ingredients.iterrows():
    ingredient = row['pizza_ingredients']
    ingredient_qty = row['items_qty']

    # Calculate the required quantity of the ingredient
    required_quantity = predicted_quantity * ingredient_qty

    # Add the required quantity to the dictionary
    if ingredient not in ingredient_quantities:
      ingredient_quantities[ingredient] = 0
    ingredient_quantities[ingredient] += required_quantity




In [41]:
# Create a DataFrame from the ingredient quantities
ingredient_requirements_df = pd.DataFrame.from_dict(ingredient_quantities, orient='index', columns=['required_quantity'])



In [43]:
# Print the ingredient requirements
ingredient_requirements_df

,required_quantity
Barbecued Chicken,5475.941765
Red Peppers,11352.798786
Green Peppers,8098.953672
Tomatoes,34844.716595
Red Onions,54751.007538
...,...
Luganega Sausage,2647.457050
Onions,1323.728525
Soppressata Salami,4867.555669
Peperoncini verdi,1434.813347


In [45]:


# Create a purchase order DataFrame
purchase_order_df = ingredient_requirements_df.copy()

# Add a column for the unit of measure (assuming all ingredients are in grams)
purchase_order_df['unit'] = 'grams'

# Rename the columns for better readability
purchase_order_df = purchase_order_df.rename(columns={'required_quantity': 'quantity'})

# Add a header to the purchase order
print('Purchase Order:')
print('----------------')

# Print the purchase order table
print(purchase_order_df.to_string())


Purchase Order:
----------------
                                quantity   unit
Barbecued Chicken            5475.941765  grams
Red Peppers                 11352.798786  grams
Green Peppers                8098.953672  grams
Tomatoes                    34844.716595  grams
Red Onions                  54751.007538  grams
Barbecue Sauce               1825.313922  grams
Bacon                       20179.703943  grams
Pepperoni                   23923.636148  grams
Italian Sausage               341.952555  grams
Chorizo Sausage              1709.762775  grams
Brie Carre Cheese             262.227284  grams
Prosciutto                    262.227284  grams
Caramelized Onions                   NaN  grams
Pears                          87.409095  grams
Thyme                          43.704547  grams
Garlic                      17841.155606  grams
?duja Salami                 1612.325925  grams
Pancetta                     2418.488887  grams
Friggitello Peppers           403.081481  grams
Chicken